In [ ]:
!pip install pydub
!pip install SpeechRecognition
!pip install google-auth google-auth-oauthlib google-auth-httplib2
!pip install google-cloud-translate
!pip install webrtcvad
!pip install scipy
!pip install nltk
!pip install AudioSegment
!pip install google-cloud-speech
!pip install googletrans==3.1.0a0

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
import webrtcvad
import numpy as np
import google.auth
from google.cloud import speech_v1
from google.cloud import translate_v2 as translate
import scipy
from scipy.io import wavfile
import googletrans
googletrans.__version__
import nltk

from googletrans import Translator
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.util import ngrams
from collections import Counter
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

In [ ]:
translator=Translator()

# Example dataset of news articles
data =  pd.read_csv('/content/News.csv', encoding='latin-1')

trueArticles = data[data['Binary Label'] == 1]['Text']
fakeArticles = data[data['Binary Label'] == 0]['Text']
# news_articles = data['Text']
# print(news_articles);
# print(trueArticles)

In [ ]:
print (data)

In [ ]:
data.shape

(7591, 5)

In [ ]:
data.isnull().sum()

In [ ]:
data = data.fillna('')

In [ ]:
data.head()

In [ ]:
def transcribe_audio(audio_path,lang):
# Load audio file
  sound = AudioSegment.from_wav(audio_path)

In [ ]:
def transcribe_audio(audio_path, lang):
    if lang == 'Sinhala':
        # Convert audio to text using Google Cloud Speech API
        r = sr.Recognizer()
        with sr.AudioFile(audio_path) as source:
            audio = r.record(source)
        text = r.recognize_google(audio, language='si-LK')
        print(text)
        # translator = translate.Translator()
        result = translator.translate(text)
        text = result.text
        print(text)
        # Print the transcribed text
        print("Transcribed Text: ", text)

    elif lang == 'Tamil':
        # Convert audio to text using Google Cloud Speech API
        r = sr.Recognizer()
        with sr.AudioFile(audio_path) as source:
            audio = r.record(source)
        text = r.recognize_google(audio, language='ta-LK')
        print(text)
        # translator = translate.Translator()
        result = translator.translate(text)
        text = result.text
        print(text)
        # Print the transcribed text
        print("Transcribed Text: ", text)

    else:
        # Convert audio to text using Google Cloud Speech API
        r = sr.Recognizer()
        with sr.AudioFile(audio_path) as source:
            audio = r.record(source)
        text = r.recognize_google(audio)

        # Print the transcribed text
        print("Transcribed Text: ", text)

    return text

In [ ]:
def translate_text(text, target_language="en"):
    from google.cloud import translate

    # credentials_path = '/content/heroic-equinox-387605-2bf604959d3e.json'  
    # translate_client = translate.Client(credentials=credentials_path)
    result = translate_client.translate(text, target_language=target_language)

    return result['input'], result['translatedText']

In [ ]:
def split_array(arr, size):
    arrs = []
    while len(arr) > size:
        piece = arr[:size]
        arrs.append(piece)
        arr = arr[size:]
    arrs.append(arr)
    return arrs

In [ ]:
def is_human_audio(audio_path):
  fs, audio = wavfile.read(audio_path)
  audio_n = audio/float(2**15)

  vad = webrtcvad.Vad(3)
  def audioSlice(x, fs, framesz, hop):
      framesamp = int(framesz*fs)
      hopsamp = int(hop*fs)
      X = scipy.array([x[i:i+framesamp] for i in range(0, len(x)-framesamp, hopsamp)])
      return X
  framesz=10./1000 #10 ms 
  hop = 1.0*framesz
  Z = audioSlice(audio_n, fs, framesz, hop)
  fr = np.int16(Z[100] * 32768).tobytes()
  aud = vad.is_speech(fr, fs)  
  if (aud):
      print("Human Voice ")
      return True
  else:
      print("Generated Voice")
      return False

In [ ]:

if __name__ == "__main__":
    
   # Transcribe the audio to text
    lang = input("Is This English or Sinhala or Tamil.?")
    
    if lang=='Sinhala':
      audio_path = "/content/Sinhala voice.wav"
    elif lang=='Tamil':
      audio_path = "/content/Tamil voice.wav"
    else:
      audio_path = "/content/English voice.wav"
    text = transcribe_audio(audio_path,lang)
    
    # Detect if the audio is human or generated
    is_human = is_human_audio(audio_path)
  
    print(f"Input Text: {text}")   
  

In [ ]:
n_val_of_text=0
n_val_of_article=0

In [ ]:
def preprocess(text, n):
    global n_val_of_text
    global n_val_of_article

    # Convert text to lowercase
    text = text.lower()

    # Tokenize text into words
    words = word_tokenize(text)

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalnum() and word not in stop_words]
    
    pos = pos_tag(words)
    
    # Extract nouns and adjectives
    nouns_adj = [word[0] for word in pos if word[1] in ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS']]
    
    # Take ngrams from text
    ngram_list = ngrams(text.split(' '), n)
    nlist = list(ngram_list)

    # Adding nouns and adjectives to entire ngram utilizing parts of speach
    nadjtup = tuple(nouns_adj)
    nlist.append(nadjtup)
  
    return nlist
    


In [ ]:
# Preprocess each news article and store n-grams

def new_compare(ngramFrequency, textng):
    score = 0
    for ngs, count in ngramFrequency.items():
      for ang in textng:
        check = all(item in ngs for item in ang) # check all items in a ngram of text contain in a ngram of article text
        if check:
          score += count
    return score

In [ ]:
# text='coronavirus is bioengineered. it may use as bioweapon'
# text= 'The coronavirus was first discovered in Wuhan'

# Preprocess each news article and store n-grams for different values of n
article_ngrams_1gram = []
article_ngrams_2gram = []
article_ngrams_3gram = []

def makeListngram(ng):
      nglis=[]
      for e in ng:
        for ngms in e: #here
          nglis.append(ngms)
      return nglis

def gettingScore(trueArticles, fakeArticles, nN):
    text_ngrams = []
    # for article3 in text:
    d=preprocess(text, nN)

    # print(list(d))
    text_ngrams.append(d)    

    trueArticlesNgrams = []
    fakeArticlesNgrams = []

    for articleT in trueArticles:
      d1 = preprocess(articleT, nN)
      trueArticlesNgrams.append(d1)

    for articleF in fakeArticles:
      d2 = preprocess(articleF, nN)
      fakeArticlesNgrams.append(d2)


    listOfNgramsTrue = makeListngram(trueArticlesNgrams)
    listOfNgramsFake = makeListngram(fakeArticlesNgrams)
    # print(listOfNgramsFake[10:])

    Truengram_freq = Counter(listOfNgramsTrue)
    Fakengram_freq = Counter(listOfNgramsFake)


    listedTextngrams = text_ngrams[0] #here
    TrueScore = new_compare(Truengram_freq, listedTextngrams)
    FakeScore = new_compare(Fakengram_freq, listedTextngrams)

    return TrueScore, FakeScore


scores = []

for i in range(3,6):
  retScores = gettingScore(trueArticles, fakeArticles, i)
  scores.append(retScores)
print(scores)

# Getting total true score and total score (true+fake)
totalScore=0
totals=[]
for score in scores:
  totals.append(score[0]+score[1])
  totalScore += (score[0]+score[1])

maxScoreIndex = totals.index(max(totals))
BestNgramTrueScore = scores[maxScoreIndex][0]
totalScore = max(totals)
print(totals)

print(BestNgramTrueScore, totalScore)
if totalScore==0:
  print('Cannot predict. insufficient data.')

else:
  confidence = (BestNgramTrueScore/totalScore)*100
  print('Accuracy : ')
  print(str(confidence)+'%')
  if confidence>50:
    print('This is a Real News.')

  else:
    print('This is a Fake News.')

[(50, 0), (25, 0), (25, 0)]
[50, 25, 25]
50 50
Accuracy : 
100.0%
This is a Real News.
